# CAO Points

## Introduction

The CAO points data, available from the [CAO website](http://www.cao.ie), is published in a different format for each of the years 2019, 2020, and 2021. Each year's data, therefore, requires a different approach to acquisition, conversion to pandas DataFrame, and cleaning. The 2019 data is published in two PDF files; one for level 8 courses, and one for levels 6 and 7. The 2020 data is published as an Excel spreadsheet, and the 2021 data as preformatted text in a HTML web page.

The attributes of interest for comparison between the various years' datasets are `Course Code`, `Course Name`, `Institution Name`, `EOS`, which is the number of points achieved by the last applicant to be offered a place on the course, and `Mid`, which is the mid point between the number of points held by the highest point score and the lowest point score of the applicants offered a place on the course [1]. The 2021 data does not explicitly contain an either an `EOS` or a `Mid` column. It does provide the *Round 1* and *Round 2* points required for entry into each course as `RND1` and `RND2`. Examination of the 2020 data, which contains both an `EOS` field *and* `RND1` and `RND2` fields demonstrates that the `EOS` field is equal to the `RND2` value if it exists, otherwise the `RND1` value (```EOS = RND1 if RND1 else RND2```). As for the `Mid` field; this information does not appear to be available yet for the 2021 data.

In [2]:
# Imports
# Data analysis library
import pandas as pd
# Plotting library
import matplotlib.pyplot as plt
# PDF table parsing
from tabula import read_pdf
# Retrieval of resources from WWW
import requests
# URL construction
from requests.compat import urljoin
# Various utilities, mainly path construction
import os
# Creation of datetime strings for filenames
from datetime import datetime
# Regular expressions
import re
import numpy as np
import hashlib


## Acquiring the data


In [4]:
# Location of CAO points data
base_url = 'http://www2.cao.ie/points/'
# Local data directory
data_dir = 'data/cao'
backup_dir = 'data/cao/backup'

# Dictionary of source file names mapped to the file names that will be used locally
points_urls  = ({'l8.php'                  : 'cao_2021_lvl8.html',
                 'l76.php'                 : 'cao_2021_lvl76.html',
                 'CAOPointsCharts2020.xlsx': 'cao_2020_lvl876.xlsx'
                 })

# The rest of points_urls can be assembled programmatically
# as filenames follow a pattern

# List of years as 2-digit strings from 2019 to 2005
years = [str(i).zfill(2) for i in range(19, 4, -1)]
# For each year (2019 to 2005)
for year in years:
    levels = ('lvl8', 'lvl76')
    # For each level 
    for level in levels:
        # construct remote filename
        remote_name = level + '_' + year + '.pdf'
        # construct local filename
        local_name = 'cao_20' + year + '_' + level + '.pdf'
        # Add remote and local filenames as keys and values in points_urls dict
        points_urls[remote_name] = local_name



In [5]:

# for each of the source files 
for url in (points_urls):
    # construct url and fetch content
    response = requests.get(urljoin(base_url, url))
    
    try:
        # attempt to open any previously downloaded local file
        with open(os.path.join(data_dir, points_urls[url]), "rb") as f:
            # Calculate md5 hashes for the local file and the remote file
            md5_local = hashlib.md5(f.read()).hexdigest()
            md5_response = hashlib.md5(response.content).hexdigest()

            # Set write_flag to False if the hashes are equal and True if they are not
            write_flag = (md5_local != md5_response)
    except IOError:
        # if the local file does not exist set the write_flag to True and move on
        write_flag = True

    # If the write_flag is True
    if write_flag:
        print(f"File: {points_urls[url]} has changed since last download. Updating...")
        # split the filename into name and extension
        fname, extension = os.path.splitext(points_urls[url])
        # construct unique filename by inserting datetime string between filename and extension
        filename = fname + datetime.now().strftime("_%Y%m%d_%H%M%S") + extension

        # write the timestamped remote file to the backup directory
        with open(os.path.join(backup_dir, filename), 'wb') as f:
            f.write(response.content)

        # also write the remote file to the data directory, overwriting any previous file
        with open(os.path.join(data_dir, points_urls[url]), 'wb') as f:
            f.write(response.content)
    else:
        print(f"File: {points_urls[url]} has not changed since last download. Skipping...")


File: cao_2021_lvl8.html has not changed since last download. Skipping...
File: cao_2021_lvl76.html has not changed since last download. Skipping...
File: cao_2020_lvl876.xlsx has not changed since last download. Skipping...
File: cao_2019_lvl8.pdf has not changed since last download. Skipping...
File: cao_2019_lvl76.pdf has not changed since last download. Skipping...
File: cao_2018_lvl8.pdf has not changed since last download. Skipping...
File: cao_2018_lvl76.pdf has not changed since last download. Skipping...
File: cao_2017_lvl8.pdf has not changed since last download. Skipping...
File: cao_2017_lvl76.pdf has not changed since last download. Skipping...
File: cao_2016_lvl8.pdf has not changed since last download. Skipping...
File: cao_2016_lvl76.pdf has not changed since last download. Skipping...
File: cao_2015_lvl8.pdf has not changed since last download. Skipping...
File: cao_2015_lvl76.pdf has not changed since last download. Skipping...
File: cao_2014_lvl8.pdf has not changed 

### 2021 Points Data

In [6]:
l8 = os.path.join(data_dir, 'cao_2021_lvl8.html')
l76 = os.path.join(data_dir, 'cao_2021_lvl76.html')

# Regular expression to capture fields from lines
# Lines consist of 2 letters and 3 numbers, comprising the course code; some whitespace; 
# 50 characters which start with a non-whitespace character; some more whitespace;
# some optional non whitespace characters comprising round 1 points; some more whitespace;
# and, optionally some more non-whitespace characters comprising round 2 points if present
re_fields = re.compile('^([A-Z]{2}[0-9]{3})\s+(\S.{49})\s+(\S+)?\s+(\S+)?')

# array to hold matched groups
data = []

for datafile, level in zip((l8, l76), (8, 76)):
    # encoding=cp1252 necessary to decode some characters on page
    with open(datafile, 'r', encoding='cp1252') as f:
        for line in f:
            match = re.match(re_fields, line)
            if match:
                fields = list(match.groups())
                fields.append(level)
                data.append(fields)

# column names
columns = ['Course Code', 'Course Name', 'Rnd1', 'Rnd2', 'Level']
df = pd.DataFrame.from_records(data, columns=columns)



Create new columns to hold information currently designated by *'s and #'s in numeric columns

Create new column indicating whether the course requires a test, interview or portfolio
This is indicated by a '#' in the Rnd1 or Rnd2 column
df['Test'] = df['Rnd1'].str.contains('#', na=False) | df['Rnd2'].str.contains('#', na=False)

Create a column indicating courses where not all applicants at Rnd1 point score were offered a place
This is indicated by a '*' in the Rnd1 or Rnd2 column
df['Not All'] = df['Rnd1'].str.contains('\*', na=False) | df['Rnd2'].str.contains('\*', na=False)

Create a new column for AQA meaning All Qualified Applicants were offered a place
df['AQA'] = df['Rnd1'].str.contains('AQA', na=False) | df['Rnd2'].str.contains('AQA', na=False)

Create a new column for 'New competition for available places' which seems to be courses 
for which the points have increased in round 2. Only occurs in level 76 and is indicated 
by a 'v' in 'Rnd2' column
df['New Comp'] = df['Rnd1'].str.contains('v', na=False) | df['Rnd2'].str.contains('v', na=False)

Generate 'EOS' column. == Rnd2 if it exists, otherwise Rnd1
df['EOS'] = np.where(df['Rnd2'].isnull(), df['Rnd1'], df['Rnd2'])

Remove Non-digits from Rnd1 and Rnd2 columns and convert columns to numeric values, 
with NaNs where values are missing (errors = 'coerce')
(Because NaN is a float, the whole columns must be floats)
df['Rnd1'] = pd.to_numeric(df['Rnd1'].str.replace('[^0-9]+', '', regex=True), errors='coerce')
df['Rnd2'] = pd.to_numeric(df['Rnd2'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

In [7]:
newcols = {'Test': '#', 'Not All': '\*', 'AQA': 'AQA', 'New Comp': 'v'}

for k, v in newcols.items():
    df[k] = df['Rnd1'].str.contains(v, na=False) | df['Rnd2'].str.contains(v, na=False)

# Generate 'EOS' column. == Rnd2 if it exists, otherwise Rnd1
df['EOS'] = np.where(df['Rnd2'].isnull(), df['Rnd1'], df['Rnd2'])

# Remove Non-digits from Rnd1 and Rnd2 columns and convert columns to numeric values, 
# with NaNs where values are missing (errors = 'coerce')
# (Because NaN is a float, the whole columns must be floats)
df['EOS'] = pd.to_numeric(df['EOS'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

df.head(20)

,Course Code,Course Name,Rnd1,Rnd2,Level,Test,Not All,AQA,New Comp,EOS
0,AL801,Software Design for Virtual Reality and Gaming...,300,None,8,False,False,False,False,300.0
1,AL802,Software Design in Artificial Intelligence for...,313,None,8,False,False,False,False,313.0
2,AL803,Software Design for Mobile Apps and Connected ...,350,None,8,False,False,False,False,350.0
3,AL805,Computer Engineering for Network Infrastructur...,321,None,8,False,False,False,False,321.0
4,AL810,Quantity Surveying ...,328,None,8,False,False,False,False,328.0
5,AL811,Civil Engineering ...,None,None,8,False,False,False,False,NaN
6,AL820,Mechanical and Polymer Engineering ...,327,None,8,False,False,False,False,327.0
7,AL830,General Nursing ...,451*,444,8,False,True,False,False,444.0
8,AL832,Mental Health Nursing ...,440*,431,8,False,True,False,False,431.0
9,AL835,Pharmacology ...,356,None,8,False,False,False,False,356.0


### 2020 Points Data

1. Read Excel file using pandas.read_excel, specifying header row, desired columns, and row names
2. Create and populate 'Test', 'Not All', 'Matric', and 'AQA' columns
3. Remove all non-numeric characters from 'EOS' and 'Mid' and convert to numeric type

In [49]:
def tidy_cols(df):

    cols = ['Test', 'Not All', 'Matric', 'AQA']
    markers = ['#', '*', 'mat', 'AQA']

    for col, marker in zip(cols, markers):
        df[col] = df['EOS'].str.contains(marker, na=False, regex=False)

    for col in ('EOS', 'Mid'):
        df[col] = pd.to_numeric(df[col].str.replace('[^0-9]+', '', regex=True), errors='coerce')

    return df 

In [50]:
df2020 = pd.read_excel(os.path.join(data_dir, 'cao_2020_lvl876.xlsx'), 
                       header=10, 
                       usecols="B,C,H,I,J,L", 
                       names=['Course Name', 'Course Code', 'EOS', 'EOS *', 'Mid', 'Institution Name'],
                       converters={'EOS':str,'Mid':str})

# Asterisks usually found in EOS are in a separate col in this dataset
# Move asterisks to EOS so generic parser can be used
df2020['EOS'] = np.where(df2020['EOS *'].str.contains('*', na=False, regex=False), 
    df2020['EOS'] + '*', df2020['EOS']) 
df2020 = df2020.drop('EOS *', axis=1)

df2020 = tidy_cols(df2020)
# # Create new column indicating whether the course requires a test, interview or portfolio
# # This is indicated by a '#' in the EOS column
# df2020['Test'] = df2020['EOS'].str.contains('#', na=False, regex=False)

# # Create a column indicating courses where not all applicants at EOS point score were offered a place
# # This is indicated by a '*' in the EOS column
# df2020['Not All'] = df2020['EOS'].str.contains('*', na=False, regex=False)

# # Create a column indicating courses where a matric is required
# # This is indicated by the string '+matric' in the EOS column.
# # However, the tabula table parsing has interpreted the r in matric as a cell boundary so only 'mat' 
# # remains in the EOS column and 'ic' appears in the Mid column. The 'ic' will be dealt with next 
# df2020['Matric'] = df2020['EOS'].str.contains('mat', na=False, regex=False)

# # Remove Non-digits from EOS and Mid columns and convert columns to numeric values, with NaNs where values are missing (errors = 'coerce')
# # (Because NaN is a float, the whole columns must be floats)
# # Note that EOS and Mid contain mixed dtypes and so must be converted to string before the replace operation
# df2020['EOS'] = pd.to_numeric(df2020['EOS'].str.replace('[^0-9]+', '', regex=True), errors='coerce')
# df2020['Mid'] = pd.to_numeric(df2020['Mid'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

df2020.head(100)

,Course Name,Course Code,EOS,Mid,Institution Name,Test,Not All,Matric,AQA
0,International Business,AC120,209.0,280.0,American College,False,False,False,False
1,Liberal Arts,AC137,252.0,270.0,American College,False,False,False,False
2,"First Year Art & Design (Common Entry,portfolio)",AD101,NaN,NaN,National College of Art and Design,True,False,True,False
3,Graphic Design and Moving Image Design (portfo...,AD102,NaN,NaN,National College of Art and Design,True,False,True,False
4,Textile & Surface Design and Jewellery & Objec...,AD103,NaN,NaN,National College of Art and Design,True,False,True,False
...,...,...,...,...,...,...,...,...,...
95,Theatre and Performative Practices - 3 or 4 ye...,CK112,330.0,434.0,University College Cork (NUI),False,False,False,False
96,Criminology - 3 years or 4 years (Internationa...,CK113,423.0,463.0,University College Cork (NUI),False,False,False,False
97,Social Science (Youth and Community Work) - 3 ...,CK114,777.0,NaN,University College Cork (NUI),False,False,False,False
98,Social Work - Mature Applicants only,CK115,999.0,NaN,University College Cork (NUI),False,False,False,False


### 2019 Points Data

The 2019 points data is held in two PDF files, one for level 8 courses and one for levels 6 and 7.

1. Read using tabula.read_pdf()
2. If necessary remove unwanted rows and assign header row
3. Fix and rename headers
4. Fill in institution column
5. Remove rows without course codes
6. Create and populate 'Test', 'Not All', 'Matric', and 'AQA' columns
7. Remove all non-numeric characters from 'EOS' and 'Mid' and convert to numeric type


In [81]:
def read_cao_pdf(pdf_path, header_row=None, ):
    
    df = read_pdf(pdf_path, pages='all', multiple_tables=False)[0]

    if header_row is not None:
        df.columns = df.iloc[header_row]
        df.rename_axis(None, axis=1, inplace=True)

        # Delete rows up to header_row
        df.drop(df.index[range(0, header_row + 1)], axis=0, inplace=True)

        # Reset the index
        df.reset_index(inplace=True, drop=True)

    # Create a new column in the dataframe for institution name 
    # identify institution name rows as those containing null course codes
    # and add those institution names to the new institution column
    df['Institution'] = df[df['Course Code'].isnull()]['INSTITUTION and COURSE']
    df.rename(columns={'INSTITUTION and COURSE':'Course Name'}, inplace=True)

    # Fill empty fields in the institution column with the most recent non-na field
    df['Institution'] = df['Institution'].fillna(method='ffill')

    # Remove rows containing only institution names
    df = df[df['Course Code'].notna()]

    df = tidy_cols(df)
    
    return df


In [82]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2019_lvl8.pdf")
df8 = read_cao_pdf(pdf_path)

# Repair LM124 Course Name
df8.loc[df8['Course Code']=='LM124', 'Course Name'] += 'ce)'

df8.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,Athlone Institute of Technology,False,False,False,False
2,AL802,Software Design with Cloud Computing,301.0,306.0,Athlone Institute of Technology,False,False,False,False
3,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,Athlone Institute of Technology,False,False,False,False
4,AL805,Network Management and Cloud Infrastructure,329.0,442.0,Athlone Institute of Technology,False,False,False,False
5,AL810,Quantity Surveying,307.0,349.0,Athlone Institute of Technology,False,False,False,False


In [83]:
# read the level 6 & 7 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2019_lvl76.pdf")
df76 = read_cao_pdf(pdf_path, header_row=7)

# Repair WD177 Course Name
df76.loc[df76['Course Code']=='WD177', 'Course Name'] += 'macy.)'

df76.head()


,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL600,Software Design,205.0,306.0,Athlone Institute of Technology,False,False,False,False
2,AL601,Computer Engineering,196.0,272.0,Athlone Institute of Technology,False,False,False,False
3,AL602,Mechanical Engineering,258.0,424.0,Athlone Institute of Technology,False,False,False,False
4,AL604,Civil Engineering,252.0,360.0,Athlone Institute of Technology,False,False,False,False
5,AL630,Pharmacy Technician,306.0,366.0,Athlone Institute of Technology,False,False,False,False


#### Merge dataframes

In [85]:
# add level 8 column to both dataframes
df8['Level 8'] = True
df76['Level 8'] = False

In [86]:
# conatenate level with levels 6 & 7
df = pd.concat([df8, df76], ignore_index=True)

# Rename column names to include year   
df = df.rename({'Course Name': 'Course Name 2019', 
                'EOS': 'EOS 2019', 
                'Mid':'Mid 2019', 
                'Test':'Test 2019', 
                'Not All': 'NotAll 2019',
                'Matric': 'Matric 2019',
                'AQA': 'AQA 2019',
                'Level 8': 'Level8 2019'}, axis=1)

In [88]:
df.tail()

,Course Code,Course Name 2019,EOS 2019,Mid 2019,Institution,Test 2019,NotAll 2019,Matric 2019,AQA 2019,Level8 2019
1386,WD188,Applied Health Care,206.0,339.0,Waterford Institute of Technology,False,False,False,False,False
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,Waterford Institute of Technology,False,False,False,False,False
1388,WD206,Electronic Engineering,191.0,322.0,Waterford Institute of Technology,False,False,False,False,False
1389,WD207,Mechanical Engineering,179.0,330.0,Waterford Institute of Technology,False,False,False,False,False
1390,WD208,Manufacturing Engineering,195.0,330.0,Waterford Institute of Technology,False,False,False,False,False


In [ ]:
# export dataframe to csv
df.to_csv('data/cao/cao_points_2019.csv')

## Analysing the data

In [ ]:
df[['EOS', 'Mid']].hist(bins=50, figsize=(20, 5))

## Conclusion

## References

[1] https://www.independent.ie/life/family/learning/understanding-your-cao-course-guide-26505318.html